In [39]:
import pandas as pd 
import numpy as np

In [40]:
#choosing majority class

knn = pd.read_csv("submision_knn.csv")
catboost = pd.read_csv("submision_catboost.csv")
xgboost = pd.read_csv("submision_xgb.csv")

In [41]:
submission = knn.merge(catboost,left_on='id', right_on='id')
submission = submission.merge(xgboost,left_on='id', right_on='id')
submission.columns = ['id', 'knn','catboost','xgboost'] 

In [42]:
idval = submission['id']
submission = submission[['knn','catboost','xgboost']] 

In [43]:
submission

,knn,catboost,xgboost
0,functional,functional,functional
1,functional,functional,functional needs repair
2,functional,functional,functional
3,non functional,non functional,non functional
4,functional,functional,functional
...,...,...,...
14845,non functional,non functional,functional
14846,functional,functional,functional
14847,functional,functional,functional
14848,functional,functional,functional


In [46]:
#get odd model out
submission.loc[(submission['knn'] == submission['catboost']) & (submission['xgboost'] == submission['catboost']), 'Odd One Out'] = 'All Equal' 
submission.loc[(submission['knn'] == submission['catboost']) & (submission['xgboost'] != submission['catboost']), 'Odd One Out'] = 'xgboost' 
submission.loc[(submission['knn'] != submission['catboost']) & (submission['xgboost'] == submission['catboost']), 'Odd One Out'] = 'knn' 
submission.loc[(submission['knn'] == submission['xgboost']) & (submission['xgboost'] != submission['catboost']), 'Odd One Out'] = 'catboost' 


In [45]:
print('Odd One Out')
test = pd.DataFrame(submission['Odd One Out'].value_counts())
test['% Odd One Out'] = test['Odd One Out']/14850 
test['% Odd One Out'] = (100. * test['% Odd One Out'] / test['% Odd One Out'].sum()).round(1).astype(str) + '%'
test

Odd One Out


,Odd One Out,% Odd One Out
All Equal,8649,59.8%
knn,3425,23.7%
xgboost,1251,8.6%
catboost,1144,7.9%


In [121]:
#choose prediction by most offent occuring prediction of the 3 models
submission = submission.mode(axis=1)
submission.columns = ['status_group', 'na', 'na2']
submission = submission['status_group']

In [122]:
#format submission
submission = pd.concat([idval,submission], axis = 1)
# submission.columns = ['id','status_group']
submission.to_csv("full_submision.csv", index=False)